# Init

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Conv1D, MaxPooling1D
from keras.utils import to_categorical
from keras import optimizers, metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
import copy

In [2]:
def loadPreapreData(datafile, skiprows = 0, delimiter=","):
    dataset = pd.read_csv(datafile,delimiter=delimiter, skiprows=skiprows)
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:, -1].values

    encoder = LabelEncoder()
    encoder.fit(y)
    y_enc = encoder.transform(y)
    y_cat = to_categorical(y_enc)

    return (X, y_cat, X[0].size, y_cat[0].size)

In [3]:
def getModel(in_dim, out_dim, optimizer, metrics):
    model = Sequential()
    model.add(Conv1D(64, 3, activation='relu', input_shape=(in_dim, 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(out_dim, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=metrics)
    return model

def fitWithKfold(X, y ,model, epochs):
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)

    for train, test in kfold.split(X):
        model.fit(X[train].reshape(X[train].shape[0], X[train].shape[1], 1), y[train], epochs=epochs, batch_size=10, verbose=0)
        scores = model.evaluate(X[test].reshape(X[test].shape[0], X[test].shape[1], 1), y[test], verbose=0)
    

    # Wyświetlenie wyników walidacji krzyżowej K-fold
    for i, names in enumerate(model.metrics_names):
        print(f'Metric: {names} -> Score: {scores[i]}')


In [4]:
def getModel(in_dim, out_dim, optimizer, metrics):
    model = Sequential()
    model.add(Dense(10, input_dim=in_dim, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(out_dim, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=metrics)
    return model

def fitWithKfold(X, y ,model, epochs):
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)

    for train, test in kfold.split(X):
        model.fit(X[train], y[train], epochs=epochs, batch_size=10, verbose=0)
        scores = model.evaluate(X[test], y[test], verbose=0)
    

    # Wyświetlenie wyników walidacji krzyżowej K-fold
    for i, names in enumerate(model.metrics_names):
        print(f'Metric: {names} -> Score: {scores[i]}')

In [5]:
optimizers_dict = {
    'SGD': optimizers.SGD(),
    'RMSprop': optimizers.RMSprop(),
    'Adam': optimizers.Adam(),
    'Adadelta': optimizers.Adadelta(),
    'Adagrad': optimizers.Adagrad(),
    'Adamax': optimizers.Adamax(),
    'Nadam': optimizers.Nadam(),
    'Ftrl': optimizers.Ftrl()
}

metrics = [metrics.MeanSquaredError(), metrics.BinaryAccuracy(), metrics.CategoricalAccuracy(), 
           metrics.AUC(), metrics.Precision(), metrics.Recall(), metrics.FalsePositives(), 
           metrics.FalseNegatives(), metrics.TruePositives(), metrics.TrueNegatives()]

# Iris

In [6]:
X, y, in_dim, out_dim = loadPreapreData("iris.data")
for oprimizer_name,oprimizer in optimizers_dict.items():
    model = getModel(in_dim,out_dim,oprimizer,metrics)
    print("\nIris classyfication model for optimizer: "+ oprimizer_name)
    fitWithKfold(X, y ,model, 50)


Iris classyfication model for optimizer: SGD
Metric: loss -> Score: 0.11345051229000092
Metric: mean_squared_error -> Score: 0.02536405622959137
Metric: binary_accuracy -> Score: 0.9523809552192688
Metric: categorical_accuracy -> Score: 0.9285714030265808
Metric: auc -> Score: 0.9974489212036133
Metric: precision -> Score: 0.9285714030265808
Metric: recall -> Score: 0.9285714030265808
Metric: false_positives -> Score: 1.0
Metric: false_negatives -> Score: 1.0
Metric: true_positives -> Score: 13.0
Metric: true_negatives -> Score: 27.0

Iris classyfication model for optimizer: RMSprop
Metric: loss -> Score: 0.08848368376493454
Metric: mean_squared_error -> Score: 0.022664111107587814
Metric: binary_accuracy -> Score: 0.9523809552192688
Metric: categorical_accuracy -> Score: 0.9285714030265808
Metric: auc -> Score: 0.9974489212036133
Metric: precision -> Score: 0.9285714030265808
Metric: recall -> Score: 0.9285714030265808
Metric: false_positives -> Score: 1.0
Metric: false_negatives -> 

# Cars

In [7]:
# def loadPreapreData_Cars(datafile):
#     dataset = pd.read_csv(datafile, na_values=['?'])
    
#     X = dataset.iloc[:, :-1].values
#     y = dataset.iloc[:, -1].values
    
#     X_enc=copy.deepcopy(X)

#     for i in range(X[0].size):
#         encoder = LabelEncoder()
#         encoder.fit(X[:,i])
#         X_enc[:,i] = encoder.transform(X[:,i]).astype("float")
#         # X_enc[:,i] = X_enc[:,i]/max(X_enc[:,i])

#     encoder = LabelEncoder()
#     encoder.fit(y)
#     y_enc = encoder.transform(y)

#     y_cat = to_categorical(y_enc)

#     return (X_enc, y_cat, X_enc[0].size, y_cat[0].size)

In [8]:
# def loadPreapreData_Cars(datafile):
#     dataset = pd.read_csv(datafile, header=0, delimiter=',')
    
#     # X = dataset.iloc[:, :-1].values
#     # y = dataset.iloc[:, -1].values
    
#     dataset['vhigh'],_ = pd.factorize(dataset['vhigh'])
#     dataset['vhigh.1'],_ = pd.factorize(dataset['vhigh.1'])
#     dataset['2'],_ = pd.factorize(dataset['2'])
#     dataset['2.1'],_ = pd.factorize(dataset['2.1'])
#     dataset['small'],_ = pd.factorize(dataset['small'])
#     dataset['low'],_ = pd.factorize(dataset['low'])
#     dataset['unacc'],_ = pd.factorize(dataset['unacc'])
    
#     X = dataset.iloc[:, :-1]
#     y = dataset.iloc[:, -1]
    
    
#     # X_enc=copy.deepcopy(X)

#     # for i in range(X[0].size):
#     #     encoder = LabelEncoder()
#     #     encoder.fit(X[:,i])
#     #     X_enc[:,i] = encoder.transform(X[:,i]).astype("float")
#     #     # X_enc[:,i] = X_enc[:,i]/max(X_enc[:,i])

#     # encoder = LabelEncoder()
#     # encoder.fit(y)
#     # y_enc = encoder.transform(y)

#     # y_cat = to_categorical(y_enc)

#     return (X.to_numpy, y.to_numpy, X.shape[1], 1)
# def loadPreapreData_Cars(datafile):
#     dataset = pd.read_csv(datafile)
#     X = dataset.iloc[:, :-1].values
#     y = dataset.iloc[:, -1].values

#     encoder = LabelEncoder()
#     encoder.fit(y)
#     y_enc = encoder.transform(y)
#     y_cat = to_categorical(y_enc)

#     return (X, y_cat, X[0].size, y_cat[0].size)

# X, y, X_s, y_s = loadPreapreData_Cars("car.data")
# print(X)
# print(y)

In [9]:
# X, y, in_dim, out_dim = loadPreapreData_Cars("car.data")
# # print(X)
# model = getModel(in_dim,out_dim,"adam",["accuracy"])
# fitWithKfold(X, y ,model, 50)

# Dry_Bean

In [10]:
X, y, in_dim, out_dim = loadPreapreData("Dry_Bean_Dataset.arff",25)
for oprimizer_name,oprimizer in optimizers_dict.items():
    model = getModel(in_dim,out_dim,oprimizer,metrics)
    print("\nDryBeans classyfication model for optimizer: "+ oprimizer_name)
    fitWithKfold(X, y ,model, 10)


DryBeans classyfication model for optimizer: SGD
Metric: loss -> Score: 1.8363672494888306
Metric: mean_squared_error -> Score: 0.11821090430021286
Metric: binary_accuracy -> Score: 0.8571428656578064
Metric: categorical_accuracy -> Score: 0.2652461528778076
Metric: auc -> Score: 0.6463385224342346
Metric: precision -> Score: 0.0
Metric: recall -> Score: 0.0
Metric: false_positives -> Score: 0.0
Metric: false_negatives -> Score: 1361.0
Metric: true_positives -> Score: 0.0
Metric: true_negatives -> Score: 8166.0

DryBeans classyfication model for optimizer: RMSprop
Metric: loss -> Score: 1.8374723196029663
Metric: mean_squared_error -> Score: 0.11824189126491547
Metric: binary_accuracy -> Score: 0.8571428656578064
Metric: categorical_accuracy -> Score: 0.2652461528778076
Metric: auc -> Score: 0.6463385224342346
Metric: precision -> Score: 0.0
Metric: recall -> Score: 0.0
Metric: false_positives -> Score: 0.0
Metric: false_negatives -> Score: 1361.0
Metric: true_positives -> Score: 0.0


# Raisin

In [11]:
X, y, in_dim, out_dim = loadPreapreData("Raisin_Dataset.arff",18)
for oprimizer_name,oprimizer in optimizers_dict.items():
    model = getModel(in_dim,out_dim,oprimizer,metrics)
    print("\nRaisin classyfication model for optimizer: "+ oprimizer_name)
    fitWithKfold(X, y ,model, 20)


Raisin classyfication model for optimizer: SGD
Metric: loss -> Score: 0.693149745464325
Metric: mean_squared_error -> Score: 0.2500012218952179
Metric: binary_accuracy -> Score: 0.49438202381134033
Metric: categorical_accuracy -> Score: 0.49438202381134033
Metric: auc -> Score: 0.5
Metric: precision -> Score: 0.49438202381134033
Metric: recall -> Score: 0.49438202381134033
Metric: false_positives -> Score: 45.0
Metric: false_negatives -> Score: 45.0
Metric: true_positives -> Score: 44.0
Metric: true_negatives -> Score: 44.0

Raisin classyfication model for optimizer: RMSprop
Metric: loss -> Score: 0.8875734806060791
Metric: mean_squared_error -> Score: 0.21564702689647675
Metric: binary_accuracy -> Score: 0.7415730357170105
Metric: categorical_accuracy -> Score: 0.7415730357170105
Metric: auc -> Score: 0.8006565570831299
Metric: precision -> Score: 0.7415730357170105
Metric: recall -> Score: 0.7415730357170105
Metric: false_positives -> Score: 23.0
Metric: false_negatives -> Score: 23